In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"]="gpt-4o"

In [5]:
from pydantic import BaseModel
from typing import List

class SubSection(BaseModel):
    title:str
    content:str

class BlogPost(BaseModel):
    title:str
    introduction:str
    sections: List[SubSection]
    sources: List[str]
    hashtags: List[str]


In [4]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    verbose=True,
    tools=[
        search_tool,
        scrape_tool,
    ],
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory="""
    You write content that keeps people engage and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

task = Task(
    description="Write a blog post about {topic}",
    agent=editor,
    expected_output="A blog pos with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.",
    output_file="blog_post.md",
    output_pydantic=BlogPost,
)

crew = Crew(agents=[researcher,editor,], tasks=[task], verbose=2,)

result = crew.kickoff(
    inputs=dict(
        topic="The biggest box office flops of 2024",
    ),
)

 [DEBUG]: == Working Agent: Senior Writer/Editor
 [INFO]: == Starting Task: Write a blog post about The biggest box office flops of 2024


> Entering new CrewAgentExecutor chain...
To write an engaging blog post about the biggest box office flops of 2024, I first need to gather accurate and detailed information on the movies that were considered flops this year. This includes their budget, box office revenue, critical reception, and possible reasons why they failed.

Since I need in-depth research and factual data, I'll delegate this task to the Senior Researcher to gather all the necessary information.

Action: Delegate work to co-worker
Action Input: 
{
    "co-worker": "Senior Researcher",
    "task": "Research and compile detailed information on the biggest box office flops of 2024.",
    "context": "Please provide a list of the movies that were considered the biggest box office flops of 2024. For each movie, include the budget, box office revenue, critical reception (both from cri